In [2]:
from calcula_resultados import compute_indicators_from_files
import pandas as pd

files = ["balancetes/balancete1.json"]
result = compute_indicators_from_files(files)



{'per_file': [{'file': 'balancetes/balancete1.json', 'indicators': {'receita_bruta': 684563.19, 'impostos_vendas': 3160.22, 'receita_liquida': 681402.97, 'custos': 405696.63, 'lucro_bruto': 275706.34, 'despesas': 416239.67, 'lucro_liquido': -140533.33, 'disponibilidade_caixa': 130876.58}, 'n_leaves': 32}], 'combined_totals': {'receita_bruta': 684563.19, 'impostos_vendas': 3160.22, 'receita_liquida': 681402.97, 'custos': 405696.63, 'lucro_bruto': 275706.34, 'despesas': 416239.67, 'lucro_liquido': -140533.33, 'disponibilidade_caixa': 130876.58}}


In [ ]:
# ---- DataFrame por arquivo ----
df_balancete = pd.json_normalize(
    result["per_file"]).drop(columns=["file", "n_leaves"])


df_balancete

,indicators.receita_bruta,indicators.impostos_vendas,indicators.receita_liquida,indicators.custos,indicators.lucro_bruto,indicators.despesas,indicators.lucro_liquido,indicators.disponibilidade_caixa
0,684563.19,3160.22,681402.97,405696.63,275706.34,416239.67,-140533.33,130876.58
